In [1]:
# libraries
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
from requests.adapters import HTTPAdapter

## Scraping data

In [2]:
# Scraping of single movie
r = requests.get('https://en.wikipedia.org/wiki/The_Witch_(2015_film)')

In [3]:
# Convert to bs object
soup = bs(r.content)

# Print out the HTML 
contents = soup.prettify()

info_box = soup.find(class_ = "infobox vevent")

info_rows = info_box.find_all("tr")

for row in info_rows:
    print(row.prettify())

<tr>
 <th class="infobox-above summary" colspan="2" style="font-size: 125%; font-style: italic;">
  The Witch
 </th>
</tr>

<tr>
 <td class="infobox-image" colspan="2">
  <a class="image" href="/wiki/File:The_Witch_poster.png">
   <img alt="The Witch poster.png" data-file-height="384" data-file-width="259" decoding="async" height="326" src="//upload.wikimedia.org/wikipedia/en/thumb/b/bf/The_Witch_poster.png/220px-The_Witch_poster.png" srcset="//upload.wikimedia.org/wikipedia/en/b/bf/The_Witch_poster.png 1.5x" width="220"/>
  </a>
  <div class="infobox-caption">
   Theatrical release poster
  </div>
 </td>
</tr>

<tr>
 <th class="infobox-label" scope="row" style="white-space: nowrap; padding-right: 0.65em;">
  Directed by
 </th>
 <td class="infobox-data">
  <a href="/wiki/Robert_Eggers" title="Robert Eggers">
   Robert Eggers
  </a>
 </td>
</tr>

<tr>
 <th class="infobox-label" scope="row" style="white-space: nowrap; padding-right: 0.65em;">
  Written by
 </th>
 <td class="infobox-data"

In [4]:
#import into dictionary
movie_info = {}

# get content function
def get_content_value(row_data):
    if row_data.find("li"):
        return[li.get_text(" ", strip = True).replace("\xa0", " ") for li in row_data.find_all('li')]
    else:
        return row_data.get_text(" ", strip = True).replace("\xa0", " ")

for index, row in enumerate (info_rows):
    if index == 0:
        movie_info['title'] = row.find("th").get_text(" ", strip = True)
    elif index == 1:
        continue
    else:
        content_key = row.find('th').get_text(" ", strip = True)
        content_value = get_content_value(row.find("td"))
        movie_info[content_key] = content_value

movie_info

{'title': 'The Witch',
 'Directed by': 'Robert Eggers',
 'Written by': 'Robert Eggers',
 'Produced by': ['Jay Van Hoy',
  'Lars Knudsen',
  'Jodi Redmond',
  'Daniel Bekerman',
  'Rodrigo Teixeira'],
 'Starring': ['Anya Taylor-Joy',
  'Ralph Ineson',
  'Kate Dickie',
  'Harvey Scrimshaw',
  'Ellie Grainger',
  'Lucas Dawson'],
 'Cinematography': 'Jarin Blaschke',
 'Edited by': 'Louise Ford',
 'Music by': 'Mark Korven',
 'Production companies': ['Parts and Labor',
  'RT Features',
  'Rooks Nest Entertainment',
  'Maiden Voyage Pictures',
  'Mott Street Pictures',
  'Code Red Productions',
  'Scythia Films',
  'Pulse Films',
  'Special Projects'],
 'Distributed by': ['A24 (United States)', 'Elevation Pictures (Canada)'],
 'Release dates': ['January 27, 2015 ( 2015-01-27 ) ( Sundance )',
  'February 19, 2016 ( 2016-02-19 ) (United States)'],
 'Running time': '92 minutes [1]',
 'Countries': ['United States [2]', 'Canada [3] [4]'],
 'Language': 'Early Modern English',
 'Budget': '$4 million

In [5]:
# The same for every movie from A24 Studio films list ('https://en.wikipedia.org/wiki/List_of_A24_films')

r = requests.get('https://en.wikipedia.org/wiki/List_of_A24_films')
soup = bs(r.content)

contents = soup.prettify()

In [6]:
#get funtions
def get_content_value(row_data):
    if row_data.find("li"):
        return[li.get_text(" ", strip = True).replace("\xa0", " ") for li in row_data.find_all('li')]
    elif row_data.find("br"):
        return [text for text in row_data.stripped_strings]
    else:
        return row_data.get_text(" ", strip = True).replace("\xa0", " ")

def clean_tags(soup):
    for tag in soup.find_all(["sup", "span"]):
        tag.decompose()
        
def get_info_box(url):
    
    r = requests.get(url)
    soup = bs(r.content)
    info_box = soup.find(class_ = "infobox vevent")
    info_rows = info_box.find_all("tr")

    clean_tags(soup)  
        
    movie_info = {}
    for index, row in enumerate (info_rows):
        if index == 0:
            movie_info['title'] = row.find("th").get_text(" ", strip = True)
        else:
            header = row.find('th')
            if header:
                content_key = row.find('th').get_text(" ", strip = True)
                content_value = get_content_value(row.find("td"))
                movie_info[content_key] = content_value
                
    return movie_info


In [7]:
#getting list of movies with printing movies with no attributes
try:
    page = requests.get('https://en.wikipedia.org/wiki/List_of_A24_films')
except requests.exceptions.ConnectionError:
    r.status_code = "Connection refused"
soup = bs(r.content)
movies = soup.select('.wikitable.sortable i a')

base_path = 'https://en.wikipedia.org/'

movie_info_list = []
for index, movie in enumerate(movies):

    try:
        relative_path = movie['href']
        full_path = base_path + relative_path
        title = movie['title']
        
        movie_info_list.append(get_info_box(full_path))
        
    except Exception as e:
        print(movie.get_text())
        print(e)

Rolling Stone
'NoneType' object has no attribute 'find_all'
Wehrmacht
'NoneType' object has no attribute 'find_all'
haenyeo
'NoneType' object has no attribute 'find_all'


In [8]:
len(movie_info_list)

154

In [9]:
#save to json
import json

def save_data(title, data):
    with open(title, 'w', encoding = 'utf-8') as f:
        json.dump(data, f, ensure_ascii = False, indent = 2)

In [10]:
import json

def load_data(title):
    with open(title, encoding = 'utf-8') as f:
        return json.load(f)

In [11]:
save_data('a24_data.json', movie_info_list)

## Cleaning data

In [12]:
movie_info_list = load_data("a24_data.json")

In [13]:
# looking on columns formats and what cleaning is necessary
movie_info_list[50]

{'title': 'It Comes at Night',
 'Directed by': 'Trey Edward Shults',
 'Written by': 'Trey Edward Shults',
 'Produced by': ['David Kaplan', 'Andrea Roa'],
 'Starring': ['Joel Edgerton',
  'Christopher Abbott',
  'Carmen Ejogo',
  'Riley Keough',
  'Kelvin Harrison Jr.'],
 'Cinematography': 'Drew Daniels',
 'Edited by': ['Trey Edward Shults', 'Matthew Hannam'],
 'Music by': 'Brian McOmber',
 'Production companies': ['A24', 'Animal Kingdom'],
 'Distributed by': 'A24',
 'Release dates': ['April 29, 2017 ( Overlook Film Festival )',
  'June 9, 2017 (United States)'],
 'Running time': '91 minutes',
 'Country': 'United States',
 'Language': 'English',
 'Budget': '$2.4–5 million',
 'Box office': '$20 million'}

To clean:
- convert running time to int and drop NAs
- convert budget to one format and drop NAs
- convert box office to one format and drop NAs
- clean up references
- convert release dates to datetime


In [14]:
# running time NAs
[movie.get('Running time', 'N/A') for movie in movie_info_list]

['86 minutes',
 '90 minutes',
 '94 minutes',
 '90 minutes',
 '95 minutes',
 '90 minutes',
 '108 minutes',
 '85 minutes',
 '83 minutes',
 '102 minutes',
 '89 minutes',
 '101 minutes',
 '100 minutes',
 '95 minutes',
 '112 minutes',
 '125 minutes',
 '109 minutes',
 '97 minutes',
 '92 minutes',
 '108 minutes',
 '84 minutes',
 '98 minutes',
 '128 minutes',
 '106 minutes',
 '112 minutes',
 '108 minutes',
 '118 minutes',
 '93 minutes',
 '92 minutes',
 '94 minutes',
 '81 minutes',
 '95 minutes',
 '105 minutes',
 '118 minutes',
 '107 minutes',
 '97 minutes',
 '101 minutes',
 '101 minutes',
 '91 minutes',
 '110 minutes',
 '163 minutes',
 '111 minutes',
 '122 minutes',
 '91 minutes',
 '118 minutes',
 '99 minutes',
 '93 minutes',
 '90 minutes',
 '94 minutes',
 '107 minutes',
 '91 minutes',
 '92 minutes',
 '81 minutes',
 '101 minutes',
 '100 minutes',
 '111 minutes',
 '121 minutes',
 '94 minutes',
 '103 minutes',
 '99 minutes',
 '111 minutes',
 '117 minutes',
 '103 minutes',
 '121 minutes',
 '104 m

In [15]:
# running time into an int
def minutes_to_int(running_time):
    if running_time == "N/A":
        return None
    
    if isinstance(running_time, list):
        entry = running_time[0]
        return int(running_time[0].split(" ")[0])
    
    else:
        return int(running_time.split(" ")[0])

for movie in movie_info_list:
    movie['Running time (int)'] = minutes_to_int(movie.get('Running time', 'N/A'))

In [16]:
# Look on budget 
[movie.get('Budget', 'N/A') for movie in movie_info_list]

['N/A',
 'N/A',
 '$5 million',
 '$8.2 million',
 '$2.5 million',
 'N/A',
 ['£', '8\xa0million', '($13.3\xa0million)'],
 '$2 million',
 '$1 million',
 '12 million',
 '$2.4 million',
 '$3 million',
 'N/A',
 '$5 million',
 'N/A',
 '$20 million',
 'N/A',
 '$10 million',
 '$5 million',
 '$15 million',
 'N/A',
 '$15 million',
 '$3.4 million',
 'N/A',
 '$11.9 million',
 '$7.1 million',
 '$13 million',
 'N/A',
 '$4 million',
 '$9.6 million',
 '$30,000',
 '$5 million',
 'N/A',
 '$ 4 million',
 'N/A',
 '$3 million',
 'N/A',
 '$10,000.',
 'N/A',
 '$25 million',
 '$3.5 million',
 '$1.5 million',
 'N/A',
 '$3 million',
 '$7 million',
 'N/A',
 'N/A',
 '$7 million',
 'N/A',
 'N/A',
 '$2.4–5 million',
 '$100,000',
 'N/A',
 '$2 million',
 '$5 million',
 '$2 million',
 'N/A',
 '$10 million',
 '$10 million',
 '$6 million',
 '$5-8 million',
 'N/A',
 'N/A',
 '$8 million',
 '$8 million',
 '$3.5 million',
 'N/A',
 '$10 million',
 '$5 million',
 '$2 million',
 'N/A',
 'N/A',
 'N/A',
 'N/A',
 'N/A',
 '$1.7 mil

Budget issues:
- some N/A's (drop them)
- milion as string (delete milion and number to int)
- ranges (I will take min or avg value)
- "2 milion +" - plus to delete
- one row with '£', '8\xa0million', '($13.3\xa0million)' - just to drop

In [17]:
import re

# regex for format money strings
amounts = r"thousand|million"
number = r"\d+(,\d{3})*\.*\d*"

word_re = rf"\${number}(-|\sto\s|–)?({number})?\s({amounts})"
value_re = rf"\${number}"

#functions to money conversion from strings into flaot
def word_to_value(word):
    value_dict = {"thousand": 1000, "million": 1000000}
    return value_dict[word]

def parse_word_syntax(string):
    value_string = re.search(number, string).group()
    value = float(value_string.replace(",", ""))
    word = re.search(amounts, string, flags=re.I).group().lower()
    word_value = word_to_value(word)
    return value*word_value

def parse_value_syntax(string):
    value_string = re.search(number, string).group()
    value = float(value_string.replace(",", ""))
    return value

def money_conversion(money):
    if money == "N/A":
        return None

    if isinstance(money, list):
        money = money[0]
        
    word_syntax = re.search(word_re, money, flags=re.I)
    value_syntax = re.search(value_re, money)

    if word_syntax:
        return parse_word_syntax(word_syntax.group())

    elif value_syntax:
        return parse_value_syntax(value_syntax.group())

    else:
        return None

In [18]:
# New columns: Budget (float) and Box Office (float)
for movie in movie_info_list:
    movie['Budget (float)'] = money_conversion(movie.get('Budget', "N/A"))
    movie['Box office (float)'] = money_conversion(movie.get('Box office', "N/A"))

In [19]:
[movie.get('Budget (float)', 'N/A') for movie in movie_info_list]

[None,
 None,
 5000000.0,
 8199999.999999999,
 2500000.0,
 None,
 None,
 2000000.0,
 1000000.0,
 None,
 2400000.0,
 3000000.0,
 None,
 5000000.0,
 None,
 20000000.0,
 None,
 10000000.0,
 5000000.0,
 15000000.0,
 None,
 15000000.0,
 3400000.0,
 None,
 11900000.0,
 7100000.0,
 13000000.0,
 None,
 4000000.0,
 9600000.0,
 30000.0,
 5000000.0,
 None,
 None,
 None,
 3000000.0,
 None,
 10000.0,
 None,
 25000000.0,
 3500000.0,
 1500000.0,
 None,
 3000000.0,
 7000000.0,
 None,
 None,
 7000000.0,
 None,
 None,
 2400000.0,
 100000.0,
 None,
 2000000.0,
 5000000.0,
 2000000.0,
 None,
 10000000.0,
 10000000.0,
 6000000.0,
 5000000.0,
 None,
 None,
 8000000.0,
 8000000.0,
 3500000.0,
 None,
 10000000.0,
 5000000.0,
 2000000.0,
 None,
 None,
 None,
 None,
 None,
 1700000.0,
 None,
 None,
 2900000.0,
 None,
 None,
 None,
 8000000.0,
 None,
 3000000.0,
 9000000.0,
 3000000.0,
 None,
 None,
 None,
 None,
 11000000.0,
 None,
 None,
 None,
 None,
 19000000.0,
 2000000.0,
 None,
 None,
 None,
 2000000.0,
 

In [20]:
# Convert dates to datetime

from datetime import datetime

dates = [movie.get('Release dates', 'N/A') for movie in movie_info_list]

def clean_date(date):
    return date.split("(")[0].strip()

def date_conversion(date):
    if isinstance(date, list):
        date = date[0]
        
    if date == "N/A":
        return None
        
    date_str = clean_date(date)
    print(date_str)
    
    fmts = ["%B %d, %Y", "%d %B %Y"]
    for fmt in fmts:
        try:
            return datetime.strptime(date_str, fmt)
        except:
            pass
    return None

for date in dates:
    print(date_conversion(date))
    print()

November 15, 2012
2012-11-15 00:00:00

7 September 2012
2012-09-07 00:00:00

September 4, 2012
2012-09-04 00:00:00

May 16, 2013
2013-05-16 00:00:00

January 18, 2013
2013-01-18 00:00:00

8 September 2013
2013-09-08 00:00:00

29 August 2013
2013-08-29 00:00:00

2 September 2013
2013-09-02 00:00:00

January 17, 2014
2014-01-17 00:00:00

18 May 2014
2014-05-18 00:00:00

January 19, 2014
2014-01-19 00:00:00

September 6, 2014
2014-09-06 00:00:00

January 17, 2014
2014-01-17 00:00:00

September 10, 2014
2014-09-10 00:00:00

None

November 6, 2014
2014-11-06 00:00:00

16 October 2014
2014-10-16 00:00:00

September 6, 2014
2014-09-06 00:00:00

June 16, 2014
2014-06-16 00:00:00

16 December 2014
2014-12-16 00:00:00

24 January 2015
2015-01-24 00:00:00

None


None

January 23, 2015
2015-01-23 00:00:00

April 8, 2015
2015-04-08 00:00:00

January 24, 2015
2015-01-24 00:00:00

September 4, 2015
2015-09-04 00:00:00

April 18, 2015
2015-04-18 00:00:00

January 27, 2015
2015-01-27 00:00:00

Septemb

In [21]:
for movie in movie_info_list:
    movie['Release dates (datetime)'] = date_conversion(movie.get('Release dates', 'N/A'))

November 15, 2012
7 September 2012
September 4, 2012
May 16, 2013
January 18, 2013
8 September 2013
29 August 2013
2 September 2013
January 17, 2014
18 May 2014
January 19, 2014
September 6, 2014
January 17, 2014
September 10, 2014
November 6, 2014
16 October 2014
September 6, 2014
June 16, 2014
16 December 2014
24 January 2015

January 23, 2015
April 8, 2015
January 24, 2015
September 4, 2015
April 18, 2015
January 27, 2015
September 10, 2015
March 16, 2015
May 17, 2015
April 16, 2015
15 May 2015
September 9, 2015
January 22, 2016
September 5, 2015
September 12, 2015
January 22, 2016
May 16, 2015
15 May 2016
September 2, 2016
October 8, 2016
9 September 2016
September 12, 2015
8 September 2016
April 22, 2017
12 September 2016
April 29, 2017
January 22, 2017
January 23, 2017
May 25, 2017
September 4, 2017
May 22, 2017
22 May 2017
September 1, 2017
March 12, 2017
March 11, 2017
January 25, 2017
April 22, 2017
1 September 2017
January 18, 2018
August 31, 2017
21 May 2017
January 21, 2018

In [22]:
# Ratings from imdb, rottentomatoes and metascore
import os
import requests
import urllib

def get_omdb_info(title):
    base_url = 'http://www.omdbapi.com/?'
    parameters = {"apikey": "3fafe88c", 't': title}
    params_encoded = urllib.parse.urlencode(parameters)
    full_url = base_url + params_encoded
    return requests.get(full_url).json()

def get_rotten_tomato_score(omdb_info):
    ratings = omdb_info.get('Ratings', [])
    for rating in ratings:
        if rating['Source'] == 'Rotten Tomatoes':
            return rating['Value']
    return None


In [23]:
for movie in movie_info_list:
    title = movie['title']
    omdb_info = get_omdb_info(title)
    movie['imdb'] = omdb_info.get('imdbRating', None)
    movie['metascore'] = omdb_info.get('Metascore', None)
    movie['rotten_tomatoes'] = get_rotten_tomato_score(omdb_info)

In [24]:
# One more conversion of datetime
for movie in movie_info_list:
    current_date = movie['Release dates (datetime)']
    if current_date:
        movie['Release dates (datetime)'] = current_date.strftime('%d-%m-%Y')
    else:
        movie['Release dates (datetime)'] = None

In [25]:
df = pd.DataFrame(movie_info_list)

In [26]:
pd.set_option('display.max_columns', None)
df.head()

,title,Directed by,Written by,Produced by,Starring,Cinematography,Edited by,Music by,Production companies,Distributed by,Release dates,Running time,Country,Language,Box office,Running time (int),Budget (float),Box office (float),Release dates (datetime),imdb,metascore,rotten_tomatoes,Countries,Budget,Screenplay by,Based on,Story by,Production company,Languages,Release date,Narrated by,Yiddish,Icelandic
0,A Glimpse Inside the Mind of Charles Swan III,Roman Coppola,Roman Coppola,"[Roman Coppola, Youree Henley]","[Charlie Sheen, Katheryn Winnick, Bill Murray,...",Nick Beal,Robert Schafer,"[Liam Hayes, Roger Neill]","[American Zoetrope, The Directors Bureau]","[A24, FilmBuff]","[November 15, 2012 (Rome Film Festival), Febru...",86 minutes,United States,English,"$210,565",86.0,NaN,210565.0,15-11-2012,4.6,28,16%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Ginger & Rosa,Sally Potter,Sally Potter,"[Christopher Sheppard, Andrew Litvin]","[Elle Fanning, Alice Englert, Alessandro Nivol...",Robbie Ryan,Anders Refn,NaN,"[British Film Institute, BBC Films, Adventure ...","[Artificial Eye (United Kingdom), (United King...","[7 September 2012 (Toronto), 19 October 2012 (...",90 minutes,NaN,English,$1.7 million,90.0,NaN,1700000.0,07-09-2012,6.2,69,78%,"[United Kingdom, Denmark, Canada]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Spring Breakers,Harmony Korine,Harmony Korine,"[Chris Hanley, Jordan Gertner, David Zander, C...","[James Franco, Selena Gomez, Vanessa Hudgens, ...",Benoît Debie,Douglas Crise,"[Skrillex, Cliff Martinez]","[Muse Productions, Annapurna Pictures, Divisio...",A24,"[September 4, 2012 ( Venice ), March 22, 2013 ...",94 minutes,United States,English,$31.7 million,94.0,5000000.0,31700000.0,04-09-2012,5.3,63,67%,NaN,$5 million,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,The Bling Ring,Sofia Coppola,NaN,"[Roman Coppola, Sofia Coppola, Youree Henley]","[Emma Watson, Israel Broussard, Katie Chang, T...","[Harris Savides, Christopher Blauvelt]",Sarah Flack,"[Brian Reitzell, Daniel Lopatin]","[American Zoetrope, NALA Films, Pathé Distribu...","[A24 (United States), StudioCanal (United King...","[May 16, 2013 ( Cannes ), June 12, 2013 (Franc...",90 minutes,NaN,English,$20 million,90.0,8200000.0,20000000.0,16-05-2013,5.6,66,60%,"[United States, United Kingdom, France, German...",$8.2 million,Sofia Coppola,"[""The Suspects Wore Louboutins"", by, Nancy Jo ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,The Spectacular Now,James Ponsoldt,NaN,"[Tom McNulty, Shawn Levy, Michelle Krumm, Andr...","[Miles Teller, Shailene Woodley, Brie Larson, ...",Jess Hall,Darrin Navarro,Rob Simonsen,"[Andrew Lauren Productions, 21 Laps Entertainm...",A24,"[January 18, 2013 ( Sundance ), August 2, 2013...",95 minutes,United States,English,$6.9 million,95.0,2500000.0,6900000.0,18-01-2013,7.1,82,91%,NaN,$2.5 million,"[Scott Neustadter, Michael H. Weber]","[The Spectacular Now, by Tim Tharp]",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
# drop columns
df = df.drop(columns=['Running time', 'Release dates', 'Box office', 'Countries', 'Budget',
                     'Languages', 'Release date', 'Narrated by', 'Yiddish', 'Icelandic', 'Story by', 'Production company'])

In [28]:
# save to csv
df.to_csv('a24_data_final.csv', index = False)

In [29]:
df.head()

,title,Directed by,Written by,Produced by,Starring,Cinematography,Edited by,Music by,Production companies,Distributed by,Country,Language,Running time (int),Budget (float),Box office (float),Release dates (datetime),imdb,metascore,rotten_tomatoes,Screenplay by,Based on
0,A Glimpse Inside the Mind of Charles Swan III,Roman Coppola,Roman Coppola,"[Roman Coppola, Youree Henley]","[Charlie Sheen, Katheryn Winnick, Bill Murray,...",Nick Beal,Robert Schafer,"[Liam Hayes, Roger Neill]","[American Zoetrope, The Directors Bureau]","[A24, FilmBuff]",United States,English,86.0,NaN,210565.0,15-11-2012,4.6,28,16%,NaN,NaN
1,Ginger & Rosa,Sally Potter,Sally Potter,"[Christopher Sheppard, Andrew Litvin]","[Elle Fanning, Alice Englert, Alessandro Nivol...",Robbie Ryan,Anders Refn,NaN,"[British Film Institute, BBC Films, Adventure ...","[Artificial Eye (United Kingdom), (United King...",NaN,English,90.0,NaN,1700000.0,07-09-2012,6.2,69,78%,NaN,NaN
2,Spring Breakers,Harmony Korine,Harmony Korine,"[Chris Hanley, Jordan Gertner, David Zander, C...","[James Franco, Selena Gomez, Vanessa Hudgens, ...",Benoît Debie,Douglas Crise,"[Skrillex, Cliff Martinez]","[Muse Productions, Annapurna Pictures, Divisio...",A24,United States,English,94.0,5000000.0,31700000.0,04-09-2012,5.3,63,67%,NaN,NaN
3,The Bling Ring,Sofia Coppola,NaN,"[Roman Coppola, Sofia Coppola, Youree Henley]","[Emma Watson, Israel Broussard, Katie Chang, T...","[Harris Savides, Christopher Blauvelt]",Sarah Flack,"[Brian Reitzell, Daniel Lopatin]","[American Zoetrope, NALA Films, Pathé Distribu...","[A24 (United States), StudioCanal (United King...",NaN,English,90.0,8200000.0,20000000.0,16-05-2013,5.6,66,60%,Sofia Coppola,"[""The Suspects Wore Louboutins"", by, Nancy Jo ..."
4,The Spectacular Now,James Ponsoldt,NaN,"[Tom McNulty, Shawn Levy, Michelle Krumm, Andr...","[Miles Teller, Shailene Woodley, Brie Larson, ...",Jess Hall,Darrin Navarro,Rob Simonsen,"[Andrew Lauren Productions, 21 Laps Entertainm...",A24,United States,English,95.0,2500000.0,6900000.0,18-01-2013,7.1,82,91%,"[Scott Neustadter, Michael H. Weber]","[The Spectacular Now, by Tim Tharp]"
